In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/training/train.csv


In [2]:
!pip install -q transformers accelerate bitsandbytes torch

import torch
import pandas as pd
import numpy as np
import re
import json
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from tqdm.auto import tqdm
from functools import lru_cache

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 31.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━

In [ ]:
tqdm.pandas()

def initialize_model():
    """
    Initializes and returns the LLM and tokenizer with 4-bit quantization.
    This function sets up the model to run efficiently on a Colab T4 GPU.
    """
    model_id = "mistralai/Mistral-7B-Instruct-v0.3"

    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    tokenizer = AutoTokenizer.from_pretrained(model_id)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=quantization_config,
        device_map="auto",
    )
    return model, tokenizer


def build_llm_prompt(description: str, tokenizer) -> str:
    """
    Constructs the full few-shot prompt for the LLM.

    Args:
        description (str): The product description text.
        tokenizer: The loaded Hugging Face tokenizer.

    Returns:
        str: The complete, formatted prompt ready for the LLM.
    """
    system_prompt = f"""You are an expert e-commerce data analyst. Your task is to analyze a product description and extract structured information. You must return your response ONLY as a single, valid JSON object and nothing else.

                    Extraction Rules:
                    1.  product_category: Classify the product into a "Primary-Secondary" format (e.g., "Food-Snacks", "Electronics-Headphones").
                    2.  concise_summary: Provide a brief summary including brand, product type, and key features. If no brand is mentioned, omit it. Key features must be the characteristics of what distinguishes or makes the product unique.
                    3.  premium_rating: Assign a premium score from 1 (basic/generic) to 5 (luxury/high-end) based on the description.
                    4.  quantity: Extract the numerical value. If it's not present or clear, return null. Do not convert the value (e.g., if it's 5000 milligrams, keep the quantity as 5000).
                    5.  categorized_unit: Analyze the unit and return it in a "category:standardized_unit" string format. Decide the category based on the product and unit.
                        * Categories must be one of: solid, fluid_volume, count, area, length, unknown.Some examples are mentioned
                        STANDARDIZED_UNITS_LIST = [
                        # Solid Weight:"gram", "kilogram", "ounce", "pound",
                        # Fluid Volume :"milliliter", "liter", "fluid_ounce",
                        # Count / Piece : "piece", "pack", "bottle", "can", "bag", "box", "jar", "k-cup", "capsule",
                        # Area & Length : "sq_ft", "foot",
                        # Undetermined"undetermined"]
                        * Standardize the unit: Convert variations to a common form (e.g., If it is'milligram' or 'mg' becomes solid:milligram;'Fl.Oz.' becomes fluid_volume:fluid_ounce).
                        * If a unit is a container or packaging type (e.g., 'Tea Bags', 'Carton'), use the count category and keep a clean unit name (e.g., count:tea_bag).
                        * If the unit is not present or nonsensical (e.g., '-', 'NA'), use unknown:undetermined, unless the product type clearly implies a unit (e.g., headphones are count:piece)."""

    few_shot_examples = """
    EXAMPLE 1
    Description:
    Indulge in the rich, creamy taste of Hershey's Milk Chocolate Bar. Made with farm-fresh milk, this classic American treat is perfect for s'mores or a simple snack. Each bar is crafted with care for a melt-in-your-mouth experience.
    Value : 1.55
    Unit : Oz
    
    JSON Output:
    {
      "product_category": "Food-Chocolate",
      "concise_summary": "Brand: Hershey's. Type: Milk Chocolate Bar. Features: Classic American treat, made with farm-fresh milk.",
      "premium_rating": 3,
      "quantity": 1.55,
      "categorized_unit": "solid:ounce"
    }
    ---
    EXAMPLE 2
    Description:
    Experience true wireless freedom with the new AuraSound Pro 3 earbuds. Featuring advanced noise-cancellation technology, 24-hour battery life with the charging case, and crystal-clear audio. Bluetooth 5.3 provides a seamless connection.
    Value : 1
    Unit : NA
    
    JSON Output:
    {
      "product_category": "Electronics-Earbuds",
      "concise_summary": "Brand: AuraSound. Type:Wireless Earbuds. Features: Advanced noise-cancellation, 24-hour battery, Bluetooth 5.3.",
      "premium_rating": 4,
      "quantity": 1,
      "categorized_unit": "count:piece"
    }
    ---
    EXAMPLE 3
    Description:
    Pure Creatine Monohydrate powder for muscle growth. Lab tested for purity and potency.
    Value : 5000
    Unit : milligram
    
    JSON Output:
    {
      "product_category": "Health-Supplements",
      "concise_summary": "Type: Creatine Monohydrate powder. Features: Lab tested for purity.",
      "premium_rating": 3,
          "quantity": 5000,
      "categorized_unit": "solid:milligram"
    }
    ---
    EXAMPLE 4
    Description:
    High quality cleaning liquid. Works great on all surfaces. Get yours today!
    Value : NA
    Unit : NA
    
    JSON Output:
    {
      "product_category": "Household-Cleaning",
      "concise_summary": "Type: Cleaning liquid. Features: Works on all surfaces.",
      "premium_rating": 2,
      "quantity": null,
      "categorized_unit": "unknown:undetermined"
    }
    """

    user_prompt = f"""
    *Description:*
    {description}
    
    *JSON Output:*
    """

    messages = [
        {"role": "user", "content": system_prompt + few_shot_examples + user_prompt}
    ]

    final_prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return final_prompt


def get_structured_data_from_llm(prompt: str, model, tokenizer) -> dict:
    """
    Sends a prompt to the LLM and parses the JSON response.

    Args:
        prompt (str): The fully formatted prompt.
        model: The loaded Hugging Face model.
        tokenizer: The loaded Hugging Face tokenizer.

    Returns:
        dict: A dictionary containing the extracted features, or a default dict on failure.
    """
    default_response = {
        "product_category": None, "concise_summary": None, "premium_rating": None,
        "quantity": None, "unit": None
    }

    try:
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=4096).to("cuda")
        input_token_length = inputs.input_ids.shape[1]

        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.1,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

        generated_token_ids = outputs[0, input_token_length:]
        response_text = tokenizer.decode(generated_token_ids, skip_special_tokens=True)

        json_match = re.search(r'\{.*\}', response_text, re.DOTALL)

        if json_match:
            json_str = json_match.group(0)
            if json_str.strip():
                return json.loads(json_str)
            else:
                print("Warning: Empty JSON string found.")
                return default_response
        else:
            print(f"Warning: No valid JSON object found in model response:\n{response_text}")
            return default_response

    except Exception as e:
        print(f"An error occurred during LLM inference or JSON parsing: {e}")
        return default_response
data = pd.read_csv('/kaggle/input/training/train.csv')
df = pd.DataFrame(data[10000:15000])

model, tokenizer = initialize_model()

def augment_row(row):
    description = row['catalog_content']
    prompt = build_llm_prompt(description, tokenizer)
    structured_data = get_structured_data_from_llm(prompt, model, tokenizer)

    value_pattern = re.compile(r"Value\s*:\s*(.*)")
    unit_pattern = re.compile(r"Unit\s*:\s*(.*)")

    # --- Initialize variables ---
    extracted_value = None
    extracted_unit = None

    # --- Process the last two lines ---
    # Split the text into lines and get the last two non-empty lines
    lines = [line for line in description.strip().split('\n') if line.strip()][-2:]

    for line in lines:
        value_match = value_pattern.search(line)
        unit_match = unit_pattern.search(line)
        
        # If the value pattern matches, extract and strip the result
        if value_match:
            # group(1) gets the content from the parentheses (.*)
            extracted_value = value_match.group(1).strip()
            
        # If the unit pattern matches, extract and strip the result
        if unit_match:
            extracted_unit = unit_match.group(1).strip()
    structured_data["regex_unit"] = extracted_unit
    structured_data["regex_value"] = extracted_value

    
    return pd.Series(structured_data)

augmented_data = df.progress_apply(augment_row, axis=1)
df_augmented = pd.concat([df, augmented_data], axis=1)

df_augmented['quantity'] = pd.to_numeric(df_augmented['quantity'], errors='coerce')
df_augmented['price_per_unit'] = df_augmented['price'] / df_augmented['quantity']
df_augmented['price_per_unit'].replace([np.inf, -np.inf], np.nan, inplace=True)
df_augmented.to_csv('augmented_data_3.csv', index=False)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

In [ ]:
df_augmented